In [154]:
import pandas as pd
from pybaseball import statcast
from pybaseball import schedule_and_record
from datetime import datetime
import time
pd.options.display.max_rows=100
pd.options.display.max_columns=100

In [46]:
df1=pd.read_csv("abs_5-11.csv")
df2=pd.read_csv("abs_6-24.csv")

In [60]:
df1.shape,df2.shape

((38831, 89), (40000, 89))

In [61]:
df=pd.concat([df1,df2],axis=0).reset_index(drop=True)

In [62]:
df.player_name.nunique()

777

In [63]:
df.events.value_counts()

field_out                    34974
strikeout                    19678
single                       12281
double                        3980
home_run                      2635
force_out                     1750
grounded_into_double_play     1682
sac_fly                        564
triple                         401
sac_bunt                       383
double_play                    213
fielders_choice_out            149
strikeout_double_play           86
fielders_choice                 47
sac_fly_double_play              6
triple_play                      1
sac_bunt_double_play             1
Name: events, dtype: int64

In [64]:
df.events.isnull().sum()

0

In [65]:
df['hit']=df.events.apply(lambda x: 1 if x in ["single",'double','home_run','triple'] else 0)
df['ab']=df.events.apply(lambda x: 0 if x in ["sac_fly",'sac_bunt','sac_fly_double_play','sac_bunt_double_play'] else 1)

In [66]:
df['date']=pd.to_datetime(df['game_date'],infer_datetime_format=True)
sorted(df.date.unique())

[numpy.datetime64('2018-03-29T00:00:00.000000000'),
 numpy.datetime64('2018-03-30T00:00:00.000000000'),
 numpy.datetime64('2018-03-31T00:00:00.000000000'),
 numpy.datetime64('2018-04-01T00:00:00.000000000'),
 numpy.datetime64('2018-04-02T00:00:00.000000000'),
 numpy.datetime64('2018-04-03T00:00:00.000000000'),
 numpy.datetime64('2018-04-04T00:00:00.000000000'),
 numpy.datetime64('2018-04-05T00:00:00.000000000'),
 numpy.datetime64('2018-04-06T00:00:00.000000000'),
 numpy.datetime64('2018-04-07T00:00:00.000000000'),
 numpy.datetime64('2018-04-08T00:00:00.000000000'),
 numpy.datetime64('2018-04-09T00:00:00.000000000'),
 numpy.datetime64('2018-04-10T00:00:00.000000000'),
 numpy.datetime64('2018-04-11T00:00:00.000000000'),
 numpy.datetime64('2018-04-12T00:00:00.000000000'),
 numpy.datetime64('2018-04-13T00:00:00.000000000'),
 numpy.datetime64('2018-04-14T00:00:00.000000000'),
 numpy.datetime64('2018-04-15T00:00:00.000000000'),
 numpy.datetime64('2018-04-16T00:00:00.000000000'),
 numpy.datet

In [67]:
df['htd']=df.sort_values(['date','inning'],ascending=True).groupby(['player_name']).hit.cumsum()
df['abtd']=df.sort_values(['date','inning'],ascending=True).groupby(['player_name']).ab.cumsum()
df['avg']=df.htd/df.abtd

In [68]:
df[['player_name','date','inning','hit','ab','htd','abtd','avg']].sort_values(['player_name','date','inning'],ascending=True).head(100)

player_name       date  inning  hit  ab  htd  abtd       avg
34342     A.J. Cole 2018-04-03       2    1   1    1     1  1.000000
34191     A.J. Cole 2018-04-03       3    0   1    1     2  0.500000
27480     A.J. Cole 2018-04-11       3    0   0    1     2  0.500000
27261     A.J. Cole 2018-04-11       5    0   1    1     3  0.333333
35206    A.J. Ellis 2018-04-02       2    0   1    0     1  0.000000
34810    A.J. Ellis 2018-04-02       6    0   1    0     2  0.000000
34643    A.J. Ellis 2018-04-02       8    0   1    0     3  0.000000
34333    A.J. Ellis 2018-04-03       2    0   1    0     4  0.000000
34102    A.J. Ellis 2018-04-03       5    1   1    1     5  0.200000
33956    A.J. Ellis 2018-04-03       6    0   1    1     6  0.166667
31349    A.J. Ellis 2018-04-07       3    1   1    2     7  0.285714
31135    A.J. Ellis 2018-04-07       5    0   0    2     7  0.285714
30835    A.J. Ellis 2018-04-07       7    0   1    2     8  0.250000
30572    A.J. Ellis 2018-04-07      10    0   1    2     9  0.222222
27444    A.J. Ellis 2018-04-11       3    0   1    2    10  0.200000
24911    A.J. Ellis 2018-04-14       2    0   1    2    11  0.181818
24677    A.J. Ellis 2018-04-14       5    1   1    3    12  0.250000
24437    A.J. Ellis 2018-04-14       7    0   1    3    13  0.230769
22836    A.J. Ellis 2018-04-17       2    0   1    3    14  0.214286
22545    A.J. Ellis 2018-04-17       5    0   1    3    15  0.200000
22319    A.J. Ellis 2018-04-17       7    0   1    3    16  0.187500
17187    A.J. Ellis 2018-04-23       2    0   1    3    17  0.176471
17053    A.J. Ellis 2018-04-23       4    0   1    3    18  0.166667
16900    A.J. Ellis 2018-04-23       7    0   1    3    19  0.157895
16747    A.J. Ellis 2018-04-23       8    0   1    3    20  0.150000
11552    A.J. Ellis 2018-04-29       2    1   1    4    21  0.190476
11319    A.J. Ellis 2018-04-29       4    0   1    4    22  0.181818
11022    A.J. Ellis 2018-04-29       6    0   1    4    23  0.173913
10815    A.J. Ellis 2018-04-29       8    1   1    5    24  0.208333
10494    A.J. Ellis 2018-04-30       4    0   1    5    25  0.200000
10335    A.J. Ellis 2018-04-30       6    0   0    5    25  0.200000
10189    A.J. Ellis 2018-04-30       8    1   1    6    26  0.230769
8710     A.J. Ellis 2018-05-02       3    0   1    6    27  0.222222
8404     A.J. Ellis 2018-05-02       5    0   1    6    28  0.214286
4926     A.J. Ellis 2018-05-06       2    0   1    6    29  0.206897
4699     A.J. Ellis 2018-05-06       4    0   1    6    30  0.200000
4477     A.J. Ellis 2018-05-06       6    1   1    7    31  0.225806
4233     A.J. Ellis 2018-05-06       8    1   1    8    32  0.250000
2450     A.J. Ellis 2018-05-09       2    1   1    9    33  0.272727
2237     A.J. Ellis 2018-05-09       4    1   1   10    34  0.294118
2038     A.J. Ellis 2018-05-09       6    0   1   10    35  0.285714
1867     A.J. Ellis 2018-05-09       8    0   1   10    36  0.277778
727      A.J. Ellis 2018-05-11       2    0   1   10    37  0.270270
270      A.J. Ellis 2018-05-11       6    1   1   11    38  0.289474
76553    A.J. Ellis 2018-05-14       3    0   1   11    39  0.282051
76453    A.J. Ellis 2018-05-14       4    0   1   11    40  0.275000
76307    A.J. Ellis 2018-05-14       6    1   1   12    41  0.292683
76143    A.J. Ellis 2018-05-14       9    0   1   12    42  0.285714
71419    A.J. Ellis 2018-05-20       3    0   1   12    43  0.279070
71191    A.J. Ellis 2018-05-20       5    1   1   13    44  0.295455
70924    A.J. Ellis 2018-05-20       7    0   1   13    45  0.288889
70666    A.J. Ellis 2018-05-20       9    1   1   14    46  0.304348
70515    A.J. Ellis 2018-05-21       2    1   1   15    47  0.319149
70358    A.J. Ellis 2018-05-21       4    0   1   15    48  0.312500
70149    A.J. Ellis 2018-05-21       7    0   1   15    49  0.306122
70024    A.J. Ellis 2018-05-21       9    0   1   15    50  0.300000
66357    A.J. Ellis 2018-05-26       2    1   1   16    51  0.313725
66060 

In [69]:
df['player_team']=df.apply(lambda x: x.home_team if x.inning_topbot=="Bot" else x.away_team,axis=1)

In [198]:
teams=df.player_team.unique()
fixers={"WSH":"WSN","CWS":"CHW"}
teams_fixed=[x if x not in fixers.keys() else fixers[x] for x in teams]

In [200]:
team_schedule={}
missed=[]
for t in teams_fixed:
    try:
        d=schedule_and_record(2018,t)
        d['fix_date']=d.Date.str.replace("\(\d\)","").str.strip() + " 2018"
        d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)
        d['place']=d.Home_Away.apply(lambda x: "Home" if x=="Home" else "Away")
        d2=d[d.game_date>=datetime.today()][['Tm','place',"Opp","game_date"]]
        team_schedule[t]=d2
    except ValueError:
        print(t)
        missed.append(t)

In [169]:
for r in missed:
    try:
        d=schedule_and_record(2018,r)
        d['fix_date']=d.Date + " 2018"
        d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)
        d['place']=d.Home_Away.apply(lambda x: "Home" if x=="Home" else "Away")
        d2=d[d.game_date>=datetime.today()][['Tm','place',"Opp","game_date"]]
        team_schedule[r]=d2
    except ValueError:
        print(r)

CWS
CLE
TOR
CHC
KC
SF
PIT
LAD
MIN
NYY
CIN
OAK
ATL
NYM
TB
BAL
WSH
SEA
DET


In [184]:
missing_schedules={}
for r in missed:
    try:
        d=schedule_and_record(2018,r)
        missing_schedules[r]=d
    except ValueError:
        print(r)

CWS
WSH


In [181]:
for r in missing_schedules.keys():
    d=missing_schedules[r]
    d['fix_date']=d.Date.str.replace("\(\d\)","").str.strip() + " 2018"
    try:
        d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)
    except ValueError:
        print("didn't work for {}".format(r))

didn't work for KC
didn't work for DET
didn't work for CHC
didn't work for MIN
didn't work for TOR
didn't work for CLE
didn't work for ATL
didn't work for TB
didn't work for OAK
didn't work for CIN
didn't work for LAD
didn't work for NYY
didn't work for SEA
didn't work for NYM
didn't work for BAL
didn't work for PIT
didn't work for SF


In [194]:
missing

[array([u'WSN', u'BAL', u'PIT', u'NYM', u'MIA', u'SDP', u'LAD', u'CIN',
        u'BOS', u'ARI', u'TOR', u'CHC', u'ATL', u'COL'], dtype=object),
 array([u'CIN', u'MIN', u'ATL', u'MIA', u'PIT', u'LAD', u'WSN', u'SFG',
        u'COL', u'SDP', u'CHC', u'STL', u'DET'], dtype=object),
 array([u'ATL', u'ARI', u'SFG', u'CHW', u'CIN', u'CHC', u'COL', u'PIT',
        u'MIA', u'KCR', u'WSN', u'MIL', u'LAD', u'DET'], dtype=object),
 array([u'NYY', u'WSN', u'KCR', u'TEX', u'TOR', u'DET', u'BAL', u'MIN',
        u'PHI', u'TBR', u'CLE', u'MIA', u'CHW', u'ATL', u'HOU', u'NYM'], dtype=object),
 array([u'BAL', u'SEA', u'LAD', u'HOU', u'CHW', u'TBR', u'CLE', u'DET',
        u'OAK', u'SDP', u'TEX', u'ARI', u'COL'], dtype=object),
 array([u'CHW', u'HOU', u'DET', u'BOS', u'BAL', u'CLE', u'OAK', u'ARI',
        u'SEA', u'NYY', u'LAA', u'SFG', u'LAD', u'MIN', u'SDP', u'TBR'], dtype=object),
 array([u'NYM', u'TBR', u'WSN', u'MIL', u'PHI', u'ATL', u'STL', u'NYY',
        u'BOS', u'TOR', u'PIT', u'CIN'], dtype=o

In [196]:
schedule_and_record(2018,"CHW")

Date   Tm Home_Away  Opp   W/L     R    RA   Inn    W-L  \
1        Thursday, Mar 29  CHW         @  KCR     W  14.0   7.0   9.0    1-0   
2        Saturday, Mar 31  CHW         @  KCR     W   4.0   3.0   9.0    2-0   
3           Monday, Apr 2  CHW         @  TOR     L   2.0   4.0   9.0    2-1   
4          Tuesday, Apr 3  CHW         @  TOR     L   5.0  14.0   9.0    2-2   
5        Wednesday, Apr 4  CHW         @  TOR     W   4.0   3.0   9.0    3-2   
6         Thursday, Apr 5  CHW      Home  DET     L   7.0   9.0  10.0    3-3   
7         Saturday, Apr 7  CHW      Home  DET     L   1.0   6.0   9.0    3-4   
8           Sunday, Apr 8  CHW      Home  DET     L   0.0   1.0   9.0    3-5   
9           Monday, Apr 9  CHW      Home  TBR     L   4.0   5.0   9.0    3-6   
10        Tuesday, Apr 10  CHW      Home  TBR     L   5.0   6.0   9.0    3-7   
11      Wednesday, Apr 11  CHW      Home  TBR     W   2.0   1.0   9.0    4-7   
12       Thursday, Apr 12  CHW         @  MIN     L   0.0   4.0   9.0    4-8   
13         Monday, Apr 16  CHW         @  OAK     L   1.0   8.0   9.0    4-9   
14        Tuesday, Apr 17  CHW         @  OAK     L   2.0  10.0   9.0   4-10   
15      Wednesday, Apr 18  CHW         @  OAK  L-wo  11.0  12.0  14.0   4-11   
16         Friday, Apr 20  CHW      Home  HOU     L   0.0  10.0   9.0   4-12   
17       Saturday, Apr 21  CHW      Home  HOU     L   1.0  10.0   9.0   4-13   
18         Sunday, Apr 22  CHW      Home  HOU     L   1.0   7.0   9.0   4-14   
19         Monday, Apr 23  CHW      Home  SEA     W  10.0   4.0   9.0   5-14   
20        Tuesday, Apr 24  CHW      Home  SEA     L   0.0   1.0   9.0   5-15   
21      Wednesday, Apr 25  CHW      Home  SEA     L   3.0   4.0   9.0   5-16   
22       Thursday, Apr 26  CHW         @  KCR     W   6.0   3.0   9.0   6-16   
23         Friday, Apr 27  CHW         @  KCR     W   7.0   4.0  11.0   7-16   
24   Saturday, Apr 28 (1)  CHW         @  KCR     W   8.0   0.0   9.0   8-16   
25   Saturday, Apr 28 (2)  CHW         @  KCR     L   2.0   5.0   9.0   8-17   
26         Sunday, Apr 29  CHW         @  KCR     L   4.0   5.0   9.0   8-18   
27         Tuesday, May 1  CHW         @  STL  L-wo   2.0   3.0   9.0   8-19   
28       Wednesday, May 2  CHW         @  STL     L   2.0   3.0   9.0   8-20   
29        Thursday, May 3  CHW      Home  MIN  W-wo   6.0   5.0   9.0   9-20   
30          Friday, May 4  CHW      Home  MIN     L   4.0   6.0   9.0   9-21   
31        Saturday, May 5  CHW      Home  MIN     L   4.0   8.0   9.0   9-22   
32          Sunday, May 6  CHW      Home  MIN     L   3.0   5.0   9.0   9-23   
33         Tuesday, May 8  CHW      Home  PIT     L   6.0  10.0   9.0   9-24   
34       Wednesday, May 9  CHW      Home  PIT     L   5.0   6.0   9.0   9-25   
35         Friday, May 11  CHW         @  CHC     L   2.0  11.0   9.0   9-26   
36       Saturday, May 12  CHW         @  CHC     L   4.0   8.0   9.0   9-27   
37         Sunday, May 13  CHW         @  CHC     W   5.0   3.0   9.0  10-27   
38        Tuesday, May 15  CHW         @  PIT     L   0.0   7.0   9.0  10-28   
39      Wednesday, May 16  CHW         @  PIT     L   2.0   3.0   9.0  10-29   
40       Thursday, May 17  CHW      Home  TEX     W   4.0   2.0   9.0  11-29   
41         Friday, May 18  CHW      Home  TEX     L   5.0  12.0   9.0  11-30   
42       Saturday, May 19  CHW      Home  TEX     W   5.0   3.0   9.0  12-30   
43         Sunday, May 20  CHW      Home  TEX     W   3.0   0.0   9.0  13-30   
44         Monday, May 21  CHW      Home  BAL     L   2.0   3.0   9.0  13-31   
45        Tuesday, May 22  CHW      Home  BAL     W   3.0   2.0   9.0  14-31   
46      Wednesday, May 23  CHW      Home  BAL     W  11.0   1.0   9.0  15-31   
47       Thursday, May 24  CHW      Home  BAL     L   3.0   9.0   9.0  15-32   
48         Friday, May 25  CHW         @  DET     L   4.0   5.0   9.0  15-33   
49       Saturday, May 26  CHW         @  DET     W   8.0   4.0   9.0  16-33   
50         Sunday, Ma

In [147]:
set([y for x in missing for y in x])
team_dict={"CWS":"CHW",'CLE':'CLE'}

{u'ARI',
 u'ATL',
 u'BAL',
 u'BOS',
 u'CHC',
 u'CHW',
 u'CIN',
 u'CLE',
 u'COL',
 u'DET',
 u'HOU',
 u'KCR',
 u'LAA',
 u'LAD',
 u'MIA',
 u'MIL',
 u'MIN',
 u'NYM',
 u'NYY',
 u'OAK',
 u'PHI',
 u'PIT',
 u'SDP',
 u'SEA',
 u'SFG',
 u'STL',
 u'TBR',
 u'TEX',
 u'TOR',
 u'WSN'}

In [127]:
team_schedule['MIL']

Tm place  Opp  game_date
78   MIL  Home  KCR 2018-06-26
79   MIL  Home  KCR 2018-06-27
80   MIL  Away  CIN 2018-06-28
81   MIL  Away  CIN 2018-06-29
82   MIL  Away  CIN 2018-06-30
83   MIL  Away  CIN 2018-07-01
84   MIL  Home  MIN 2018-07-02
85   MIL  Home  MIN 2018-07-03
86   MIL  Home  MIN 2018-07-04
87   MIL  Home  ATL 2018-07-05
88   MIL  Home  ATL 2018-07-06
89   MIL  Home  ATL 2018-07-07
90   MIL  Home  ATL 2018-07-08
91   MIL  Away  MIA 2018-07-09
92   MIL  Away  MIA 2018-07-10
93   MIL  Away  MIA 2018-07-11
94   MIL  Away  PIT 2018-07-12
95   MIL  Away  PIT 2018-07-13
96   MIL  Away  PIT 2018-07-14
97   MIL  Away  PIT 2018-07-14
98   MIL  Away  PIT 2018-07-15
99   MIL  Home  LAD 2018-07-20
100  MIL  Home  LAD 2018-07-21
101  MIL  Home  LAD 2018-07-22
102  MIL  Home  WSN 2018-07-23
103  MIL  Home  WSN 2018-07-24
104  MIL  Home  WSN 2018-07-25
105  MIL  Away  SFG 2018-07-26
106  MIL  Away  SFG 2018-07-27
107  MIL  Away  SFG 2018-07-28
108  MIL  Away  SFG 2018-07-29
109  MIL  Away  LAD 2018-07-30
110  MIL  Away  LAD 2018-07-31
111  MIL  Away  LAD 2018-08-01
112  MIL  Away  LAD 2018-08-02
113  MIL  Home  COL 2018-08-03
114  MIL  Home  COL 2018-08-04
115  MIL  Home  COL 2018-08-05
116  MIL  Home  SDP 2018-08-07
117  MIL  Home  SDP 2018-08-08
118  MIL  Home  SDP 2018-08-09
119  MIL  Away  ATL 2018-08-10
120  MIL  Away  ATL 2018-08-11
121  MIL  Away  ATL 2018-08-12
122  MIL  Away  CHC 2018-08-14
123  MIL  Away  CHC 2018-08-15
124  MIL  Away  STL 2018-08-17
125  MIL  Away  STL 2018-08-18
126  MIL  Away  STL 2018-08-19
127  MIL  Home  CIN 2018-08-20
128  MIL  Home  CIN 2018-08-21
129  MIL  Home  CIN 2018-08-22
130  MIL  Home  PIT 2018-08-24
131  MIL  Home  PIT 2018-08-25
132  MIL  Home  PIT 2018-08-26
133  MIL  Away  CIN 2018-08-28
134  MIL  Away  CIN 2018-08-29
135  MIL  Away  CIN 2018-08-30
136  MIL  Away  WSN 2018-08-31
137  MIL  Away  WSN 2018-09-01
138  MIL  Away  WSN 2018-09-02
139  MIL  Home  CHC 2018-09-03
140  MIL  Home  CHC 2018-09-04
141  MIL  Home  CHC 2018-09-05
142  MIL  Home  SFG 2018-09-07
143  MIL  Home  SFG 2018-09-08
144  MIL  Home  SFG 2018-09-09
145  MIL  Away  CHC 2018-09-10
146  MIL  Away  CHC 2018-09-11
147  MIL  Away  CHC 2018-09-12
148  MIL  Home  PIT 2018-09-14
149  MIL  Home  PIT 2018-09-15
150  MIL  Home  PIT 2018-09-16
151  MIL  Home  CIN 2018-09-17
152  MIL  Home  CIN 2018-09-18
153  MIL  Home  CIN 2018-09-19
154  MIL  Away  PIT 2018-09-21
155  MIL  Away  PIT 2018-09-22
156  MIL  Away  PIT 2018-09-23
157  MIL  Away  STL 2018-09-24
158  MIL  Away  STL 2018-09-25
159  MIL  Away  STL 2018-09-26
160  MIL  Home  DET 2018-09-28
161  MIL  Home  DET 2018-09-29
162  MIL  Home  DET 2018-09-30

In [103]:
a=d.Date[1] + " 2018"
g=datetime.strptime(a,"%A, %b %d %Y")
print(g.strftime("%Y-%m-%d"))

2018-03-29


In [109]:
d['fix_date']=d.Date + " 2018"
d['game_date']=pd.to_datetime(d.fix_date.apply(lambda x: datetime.strptime(x,"%A, %b %d %Y")).apply(lambda x: x.strftime("%Y-%m-%d")),infer_datetime_format=True)

In [110]:
d.game_date

1     2018-03-29
2     2018-03-30
3     2018-03-31
4     2018-04-02
5     2018-04-03
6     2018-04-04
7     2018-04-05
8     2018-04-06
9     2018-04-07
10    2018-04-08
11    2018-04-09
12    2018-04-10
13    2018-04-11
14    2018-04-12
15    2018-04-13
16    2018-04-14
17    2018-04-15
18    2018-04-16
19    2018-04-17
20    2018-04-18
21    2018-04-20
22    2018-04-21
23    2018-04-22
24    2018-04-23
25    2018-04-24
26    2018-04-25
27    2018-04-27
28    2018-04-28
29    2018-04-29
30    2018-04-30
31    2018-05-01
32    2018-05-02
33    2018-05-04
34    2018-05-05
35    2018-05-06
36    2018-05-08
37    2018-05-09
38    2018-05-10
39    2018-05-11
40    2018-05-12
41    2018-05-13
42    2018-05-14
43    2018-05-15
44    2018-05-17
45    2018-05-18
46    2018-05-19
47    2018-05-20
48    2018-05-21
49    2018-05-22
50    2018-05-23
         ...    
113   2018-08-07
114   2018-08-08
115   2018-08-09
116   2018-08-10
117   2018-08-11
118   2018-08-12
119   2018-08-14
120   2018-08-

In [73]:
d['game_date']=pd.to_datetime(d['Date'],infer_datetime_format=True)
d

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1-03-29 00:00:00

In [59]:
df.to_csv("all_abs_edit.csv")

In [15]:
## statcast test
%timeit
data=statcast()

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [3]:
data.shape

(3815, 90)

In [7]:
data[data.events.notnull()].shape

(976, 90)